In [1]:
!pip install numpy==1.20

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 40.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [1056 lines of output]
      setup.py:66: RuntimeWarning: NumPy 1.20.0 may not yet support Python 3.10.
        warnings.warn(
      Running from numpy source directory.
      numpy/random/_bounded_integers.pxd.in has not changed
      numpy/random/_mt19937.pyx has not changed
      numpy/random/_generator.pyx has not changed
      numpy/random/_philox.pyx has not changed
      numpy/random/_bounded_integers.pyx.in has not changed
      numpy/random/_common.pyx has not changed
      numpy/random/_sfc64.pyx has not changed
      numpy/random/bit_gener

In [1]:
!pip install -U pandas==1.5.3

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkByExamples.com") \
    .master("local[*]") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/05 19:55:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import os
import sys
import boto3
import numpy as np
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from numpy.lib.stride_tricks import sliding_window_view

HISTORICAL_DATA_WINDOW = 14
FUTURE_PREDICTION_WINDOW = 3

save_bucket_name = ""

def create_training_frames(partition):

  partition.drop(columns=["Unnamed: 0"], inplace=True)
  partition.sort_values(by=["date"], inplace=True)
  split_index = partition.iloc[0]["keywordId"]
    
  train_frames_X, train_frames_Y = [], []
  test_frames_X, test_frames_Y = [], []
  train_frames_embed, test_frames_embed = [], []
  train_frames_decoder_input, test_frames_decoder_input = [], []

  train_size = (int)(len(partition) * 0.7)
  train_data = partition[:train_size]
  test_data = partition[train_size-HISTORICAL_DATA_WINDOW-FUTURE_PREDICTION_WINDOW+1:]
  data_columns = partition.columns

  train_data_arr, test_data_arr = np.array(train_data), np.array(test_data)
  if train_data_arr.shape[0] < HISTORICAL_DATA_WINDOW or test_data_arr.shape[0] < HISTORICAL_DATA_WINDOW:
    return 0
    
  train_data_frame_X = sliding_window_view(train_data_arr, window_shape = (HISTORICAL_DATA_WINDOW, train_data_arr.shape[1]))
  test_data_frame_X = sliding_window_view(test_data_arr, window_shape = (HISTORICAL_DATA_WINDOW, test_data_arr.shape[1]))
  train_data_frame_X = np.squeeze(train_data_frame_X)[:-FUTURE_PREDICTION_WINDOW]
  test_data_frame_X = np.squeeze(test_data_frame_X)[:-FUTURE_PREDICTION_WINDOW]
  train_data_frames_X, test_data_frames_X = train_data_frame_X.copy(), test_data_frame_X.copy()
  for frame in train_data_frames_X:
    train_frames_X.append(pd.DataFrame(frame, columns=data_columns))
  for frame in test_data_frames_X:
    test_frames_X.append(pd.DataFrame(frame, columns=data_columns))
    
#   with open("random.txt", "w") as f:
#     original_stdout = sys.stdout
#     sys.stdout = f
#     print(data_columns)
#     sys.stdout = original_stdout

  train_data_frame_Y = sliding_window_view(train_data_arr, window_shape = (FUTURE_PREDICTION_WINDOW, train_data_arr.shape[1]))
  test_data_frame_Y = sliding_window_view(test_data_arr, window_shape = (FUTURE_PREDICTION_WINDOW, test_data_arr.shape[1]))
  train_data_frame_Y = np.squeeze(train_data_frame_Y)[HISTORICAL_DATA_WINDOW:]
  test_data_frame_Y = np.squeeze(test_data_frame_Y)[HISTORICAL_DATA_WINDOW:]
  train_data_frames_Y, test_data_frames_Y = train_data_frame_Y.copy(), test_data_frame_Y.copy()
  for frame in train_data_frames_Y:
    train_frames_Y.append(pd.DataFrame(frame, columns=data_columns))
  for frame in test_data_frames_Y:
    test_frames_Y.append(pd.DataFrame(frame, columns=data_columns))
    
  embedding_columns = ["keyword_length", "keyword_num_words", "budget", "matchType", "country_code", "campaign_type", "targeting_type", "budget_type", "adFormat", "tactic", "costType"]
  
  for i in range(len(train_frames_X)):
    train_frames_embed.append(train_frames_X[i].loc[0][embedding_columns])
    train_frames_X[i].drop(columns=embedding_columns, inplace=True)

  for i in range(len(test_frames_X)):
    test_frames_embed.append(test_frames_X[i].loc[0][embedding_columns])
    test_frames_X[i].drop(columns=embedding_columns, inplace=True)
    
  for i in range(len(train_frames_X)):
    train_frames_X[i].drop(columns=["keywordId", "date"], inplace=True)
    train_frames_Y[i].drop(columns=embedding_columns, inplace=True)
    train_frames_Y[i].drop(columns=["keywordId", "date", "year", "month", "day", "dayoftheweek", "clicks", "impressions", "orders", "campaign_sales_perc", "campaign_spend_perc", "account_sales_perc", "account_spend_perc"], inplace=True)

  for i in range(len(test_frames_X)):
    test_frames_X[i].drop(columns=["keywordId", "date"], inplace=True)
    test_frames_Y[i].drop(columns=embedding_columns, inplace=True)
    test_frames_Y[i].drop(columns=["keywordId", "date", "year", "month", "day", "dayoftheweek", "clicks", "impressions", "orders", "campaign_sales_perc", "campaign_spend_perc", "account_sales_perc", "account_spend_perc"], inplace=True)
  
  for i in range(len(train_frames_Y)):
      train_frames_decoder_input.append(train_frames_Y[i]["cpc"])
      train_frames_Y[i].drop(columns=["cpc"], inplace=True)

  for i in range(len(test_frames_Y)):
      test_frames_decoder_input.append(test_frames_Y[i]["cpc"])
      test_frames_Y[i].drop(columns=["cpc"], inplace=True)
    
  train_frames_X, train_frames_Y = np.array(train_frames_X), np.array(train_frames_Y)
  test_frames_X, test_frames_Y = np.array(test_frames_X), np.array(test_frames_Y)
  train_frames_embed, test_frames_embed = np.array(train_frames_embed), np.array(test_frames_embed)
  train_frames_decoder_input, test_frames_decoder_input = np.array(train_frames_decoder_input), np.array(test_frames_decoder_input)
  
  train_frames_x_dir = "./train_frames_x"
  test_frames_x_dir = "./test_frames_x"
  train_frames_y_dir = "./train_frames_y"
  test_frames_y_dir = "./test_frames_y"
  train_frames_embed_dir = "./train_frames_embed"
  test_frames_embed_dir = "./test_frames_embed"
  train_frames_decoder_input_dir = "./train_frames_decoder_input"
  test_frames_decoder_input_dir = "./test_frames_decoder_input"
  
  if not os.path.exists(train_frames_x_dir):
    os.makedirs(train_frames_x_dir)
  if not os.path.exists(test_frames_x_dir):
    os.makedirs(test_frames_x_dir)
  if not os.path.exists(train_frames_y_dir):
    os.makedirs(train_frames_y_dir)
  if not os.path.exists(test_frames_y_dir):
    os.makedirs(test_frames_y_dir)
  if not os.path.exists(train_frames_embed_dir):
    os.makedirs(train_frames_embed_dir)
  if not os.path.exists(test_frames_embed_dir):
    os.makedirs(test_frames_embed_dir)
  if not os.path.exists(train_frames_decoder_input_dir):
    os.makedirs(train_frames_decoder_input_dir)
  if not os.path.exists(test_frames_decoder_input_dir):
    os.makedirs(test_frames_decoder_input_dir)
    
  np.save(os.path.join(train_frames_x_dir, f"par_{split_index}.npy"), train_frames_X)
  np.save(os.path.join(test_frames_x_dir, f"par_{split_index}.npy"), test_frames_X)
  np.save(os.path.join(train_frames_y_dir, f"par_{split_index}.npy"), train_frames_Y)
  np.save(os.path.join(test_frames_y_dir, f"par_{split_index}.npy"), test_frames_Y)
  np.save(os.path.join(train_frames_embed_dir, f"par_{split_index}.npy"), train_frames_X)
  np.save(os.path.join(test_frames_embed_dir, f"par_{split_index}.npy"), test_frames_X)
  np.save(os.path.join(train_frames_decoder_input_dir, f"par_{split_index}.npy"), train_frames_Y)
  np.save(os.path.join(test_frames_decoder_input_dir, f"par_{split_index}.npy"), test_frames_Y)

  return 0

In [9]:
import boto3
s3 = boto3.client('s3')
obj = s3.get_object(Bucket='training-data-lstm', Key = 'processed_training_data.csv')
dataset = pd.read_csv(obj['Body'])
data_columns = dataset.columns
spark_df = spark.createDataFrame(dataset)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.readRDDFromFile.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.api.java.JavaRDD$.readRDDFromInputStream(JavaRDD.scala:252)
	at org.apache.spark.api.java.JavaRDD$.readRDDFromFile(JavaRDD.scala:239)
	at org.apache.spark.api.python.PythonRDD$.readRDDFromFile(PythonRDD.scala:274)
	at org.apache.spark.api.python.PythonRDD.readRDDFromFile(PythonRDD.scala)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py

In [ ]:
spark_df = spark_df.pandas_api()

In [ ]:
grouped = spark_df.groupby("keywordId")

In [ ]:
df = grouped.apply(create_training_frames)

In [ ]:
df